# Libraries

In [1]:
import os
import requests
import re
from util_IO import get_use_case_main_dir
import pickle

import pandas as pd

# Settings

## Packages

In [2]:
# Set pandas to display a maximum of 300 columns
pd.set_option('display.max_columns', 300)

## Directories

Derive the directory in order to better manage files locations

In [3]:
# Directory for use case
camels_gb_use_case_dir = get_use_case_main_dir()

print(f"Main directory for use case:\t\t\t\t{camels_gb_use_case_dir}")


# Directory for dataset
camels_gb_datasets_dir = os.path.join(
    camels_gb_use_case_dir,
    "datasets"
)
print(f"Main directory for datasets:\t\t\t\t{camels_gb_datasets_dir}")


# Main directory for camels_gb - BRONZE LAYER
camels_gb_bronze_dir = os.path.join(
    camels_gb_datasets_dir,
    "camels-gb",
    "data"
)
print(f"Main directory for camels-gb bronze layer:\t\t{camels_gb_bronze_dir}")


# Main directory for camels_gb - SILVER LAYER
camels_gb_silver_dir = os.path.join(
    camels_gb_datasets_dir,
    "camels-gb-aggregated"
)
print(f"Main directory for camels-gb silver layer:\t\t{camels_gb_silver_dir}")


# __________
# Attributes

# Bronze layer 🥉
camels_gb_data_attributes_dir = os.path.join(
    camels_gb_bronze_dir
)
print(f"Directory for camels-gb bronze layer attributes:\t{camels_gb_data_attributes_dir}")


# Silver layer 🥈
camels_gb_data_attributes_aggr_dir = os.path.join(
    camels_gb_silver_dir,
    "attributes"
)
print(f"Directory for camels-gb silver layer attributes:\t{camels_gb_data_attributes_aggr_dir}")


# __________
# Timeseries

# Bronze layer 🥉
camels_gb_data_timeseries_dir = os.path.join(
    camels_gb_bronze_dir,
    "timeseries"
)
print(f"Directory for camels-gb bronze layer timeseries:\t{camels_gb_data_timeseries_dir}")


# Silver layer 🥈
camels_gb_data_timeseries_aggr_dir = os.path.join(
    camels_gb_silver_dir,
    "timeseries"
)
print(f"Directory for camels-gb silver layer timeseries:\t{camels_gb_data_timeseries_aggr_dir}")

Main directory for use case:				/home/jupyter/RDMAI_Sensing/flow-derivation-camels-gb
Main directory for datasets:				/home/jupyter/RDMAI_Sensing/flow-derivation-camels-gb/datasets
Main directory for camels-gb bronze layer:		/home/jupyter/RDMAI_Sensing/flow-derivation-camels-gb/datasets/camels-gb/data
Main directory for camels-gb silver layer:		/home/jupyter/RDMAI_Sensing/flow-derivation-camels-gb/datasets/camels-gb-aggregated
Directory for camels-gb bronze layer attributes:	/home/jupyter/RDMAI_Sensing/flow-derivation-camels-gb/datasets/camels-gb/data
Directory for camels-gb silver layer attributes:	/home/jupyter/RDMAI_Sensing/flow-derivation-camels-gb/datasets/camels-gb-aggregated/attributes
Directory for camels-gb bronze layer timeseries:	/home/jupyter/RDMAI_Sensing/flow-derivation-camels-gb/datasets/camels-gb/data/timeseries
Directory for camels-gb silver layer timeseries:	/home/jupyter/RDMAI_Sensing/flow-derivation-camels-gb/datasets/camels-gb-aggregated/timeseries


## Hydrology Data API

In [4]:
# Base URL
APIBaseURL = "https://environment.data.gov.uk"

### Functions definitions

In [5]:
# Retrieve bulk data from REST API
def api_call(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Failed to retrieve data: {response.status_code}"

# Get nrfaStationID
def get_nrfaStationID(url):
    response = api_call(url)
    if isinstance(response, dict):
        try:
            return response['items'][0]['nrfaStationID']
        except:
            return "No nrfaStationID found"
    else:
        return "Api call failed"


# Define function to get url for query time series
def get_flow_timeseries_url(station):
    return (
        f"{APIBaseURL}/hydrology/id/measures/" +
        f"{station}" + 
        "-flow-m-86400-m3s-qualified/readings.json?" + 
        "maxeq-date=2015-09-30&mineq-date=1970-10-01" # ⚠️ Dates are hardcoded ⚠️
    )

## Files

To manage files programmatically, just the extensions are needed

In [6]:
# Extensions
attributes_files_ext = ".csv"
timeseries_files_ext = ".csv"

# Regular expression pattern to extract catchmentID
catchmentID_pattern = r"CAMELS_GB_hydromet_timeseries_(.*?)_19701001-20150930"

## Fields management

### Attributes

Common fields definitions/settings

In [7]:
# Index field
attributes_index = "gauge_id"

Definition of the fields used for each ***level of aggregation***:
 - **full list of fields** (*silver layer*) - no fields selection is made:
   - a pure aggregation with all the fields available from camels-gb dataset
   - `chalk_stream_flag`
 - **fundamental** (*silver layer*) - this level includes:
   - **ONLY** fields which are calculated/derivate from others already present in the camels-gb dataset ( with the exception of `baseflow_index`, for which some variables in camels-gb could be used to calculate it)
   - `chalk_stream_flag`
   - reference for API at `environment.data.gov.uk`

##### `fundamental` level proper attributes (=API excluded)

In [8]:
# Fields to keep
attributes_fundamental_fields = {
    "hydrologic_attributes_df": [
        'baseflow_index'
    ],
    
    "soil_attributes_df": [
        "sand_perc",
        "silt_perc",
        "clay_perc",
        "organic_perc"
    ],
    
    "climatic_attributes_df": [],
    
    "topographic_attributes_df": [
        "gauge_name",
        "gauge_lat",
        "gauge_lon",
        "gauge_elev",
        "area",
        "dpsbar",
        "elev_mean",
        "elev_min",
        "elev_10",
        "elev_50",
        "elev_90",
        "elev_max"
    ],
    
    "landcover_attributes_df": [
        "dwood_perc",
        "ewood_perc",
        "grass_perc",
        "shrub_perc",
        "crop_perc",
        "urban_perc",
        "inwater_perc",
        "bares_perc"
    ],
    
    "hydrogeology_attributes_df": [],
    
    "humaninfluence_attributes_df": [
        "surfacewater_abs",
        "groundwater_abs",
        "discharges",
        "num_reservoir",
        "reservoir_cap"

    ],
    
    "hydrometry_attributes_df": [
        "bankfull_flow"
    ],

    "chalk_streams_df": [
        "chalk_stream_flag"
    ]
}

### Time series

It is not needed to select any specific field for time series data, as all of the variables are taken in consideration for the analysis, at least during the initial stages. However, it worth to be mentioned that during the process that loads the files, a new field will be added to store the catchment ID; this piece of information is indeed present in the file name only.

In [9]:
# Date field
date_field = "date"

# Label field
label_field = "discharge_vol"

# Minimum number of samples for group time series 
min_timeseries_length = 5

# Attributes aggregations

## Single files loaded as specific `pd.DataFrame` *(multiple data frames)*

In [10]:
# Data frame names list initialization (🚩 elements will be strings!)
attributes_df_names_list = []

# Loop through all files in the directory for camels-gb attributes
for filename in os.listdir(camels_gb_data_attributes_dir):
    
    # Define current full path
    path = os.path.join(camels_gb_data_attributes_dir, filename)
    
    # Check if it's a file and has the desired extension
    if os.path.isfile(path) and filename.endswith(attributes_files_ext):
    
        # Define current table name via file name extraction
        curr_df_name = f"{filename[10:-4]}_df"
        print(curr_df_name)
        attributes_df_names_list.append(curr_df_name)

        # Read the file into a DataFrame
        df = pd.read_csv(
            path,
            dtype={attributes_index: 'str'},
            index_col=attributes_index
        )
    
        # Dynamically create a variable with `curr_df_name` as name
        globals()[curr_df_name] = df
        display(df.head(3))

climatic_attributes_df


,p_mean,pet_mean,aridity,p_seasonality,frac_snow,high_prec_freq,high_prec_dur,high_prec_timing,low_prec_freq,low_prec_dur,low_prec_timing
gauge_id,,,,,,,,,,,
10002,2.29,1.26,0.55,-0.03,0.03,13.96,1.16,son,207.89,3.17,jja
10003,2.31,1.25,0.54,-0.01,0.03,13.24,1.17,son,209.89,3.20,jja
1001,2.65,1.15,0.44,-0.15,0.03,10.98,1.11,son,179.87,2.93,jja


hydrogeology_attributes_df


,inter_high_perc,inter_mod_perc,inter_low_perc,frac_high_perc,frac_mod_perc,frac_low_perc,no_gw_perc,low_nsig_perc,nsig_low_perc
gauge_id,,,,,,,,,
10002,0.0,18.53,0.0,0.0,0.00,23.72,57.67,0.0,0.08
10003,0.0,13.14,0.0,0.0,0.00,23.03,63.72,0.0,0.12
1001,0.0,10.20,0.0,0.0,2.24,0.00,87.55,0.0,0.00


landcover_attributes_df


...

,station_type,flow_period_start,flow_period_end,flow_perc_complete,bankfull_flow,structurefull_flow,q5_uncert_upper,q5_uncert_lower,q25_uncert_upper,q25_uncert_lower,q50_uncert_upper,q50_uncert_lower,q75_uncert_upper,q75_uncert_lower,q95_uncert_upper,q95_uncert_lower,q99_uncert_upper,q99_uncert_lower,quncert_meta
gauge_id,,,,,,,,,,,,,,,,,,,
10002,VA,1971-02-01,2015-09-30,99.25,100.0,NaN,NaN,NaN,11.95,-12.37,9.55,-9.40,9.35,-9.61,9.02,-9.23,8.25,-9.50,Calculated discharge uncertainties
10003,VA,1983-05-19,2015-09-30,71.93,80.0,NaN,NaN,NaN,12.58,-12.46,8.73,-8.68,11.69,-11.89,8.00,-8.06,NaN,NaN,Calculated discharge uncertainties
1001,VA,1995-11-09,2015-09-30,44.21,31.2,NaN,25.34,-25.94,17.71,-17.01,9.00,-9.56,8.84,-8.68,8.98,-8.97,9.94,-9.72,Calculated discharge uncertainties


soil_attributes_df


,sand_perc,sand_perc_missing,silt_perc,silt_perc_missing,clay_perc,clay_perc_missing,organic_perc,organic_perc_missing,bulkdens,bulkdens_missing,bulkdens_5,bulkdens_50,bulkdens_95,tawc,tawc_missing,tawc_5,tawc_50,tawc_95,porosity_cosby,porosity_cosby_missing,porosity_cosby_5,porosity_cosby_50,porosity_cosby_95,porosity_hypres,porosity_hypres_missing,porosity_hypres_5,porosity_hypres_50,porosity_hypres_95,conductivity_cosby,conductivity_cosby_missing,conductivity_cosby_5,conductivity_cosby_50,conductivity_cosby_95,conductivity_hypres,conductivity_hypres_missing,conductivity_hypres_5,conductivity_hypres_50,conductivity_hypres_95,root_depth,root_depth_missing,root_depth_5,root_depth_50,root_depth_95,soil_depth_pelletier,soil_depth_pelletier_missing,soil_depth_pelletier_5,soil_depth_pelletier_50,soil_depth_pelletier_95
gauge_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10002,40.98,0.00,35.60,0.00,23.42,0.00,5.22,0.00,1.20,0.00,0.28,1.36,1.42,116.79,0.00,55.12,110.37,235.65,43.81,0.00,42.75,43.82,44.36,51.13,0.00,44.0,47.0,81.0,1.34,0.00,1.15,1.17,2.31,1.49,0.00,1.27,1.34,3.13,1.20,0.00,0.5,1.3,1.3,13.21,0.0,1.0,2.0,41.0
10003,43.79,0.00,37.96,0.00,18.25,0.00,1.58,0.00,1.29,0.00,1.10,1.36,1.42,88.47,0.00,55.12,102.07,110.37,43.61,0.00,42.75,43.74,44.36,47.54,0.00,44.0,47.0,52.0,1.58,0.00,1.17,1.39,2.31,1.50,0.00,1.27,1.36,2.28,1.01,0.00,0.5,1.3,1.3,7.52,0.0,1.0,1.0,30.0
1001,39.25,3.16,31.97,3.16,28.79,3.16,19.32,3.16,0.74,3.16,0.28,0.28,1.38,182.04,3.16,108.92,235.65,235.65,43.86,3.16,43.60,43.60,44.36,66.45,3.16,45.0,81.0,81.0,1.16,3.16,1.15,1.15,1.38,1.87,3.16,1.27,3.13,3.13,1.30,3.16,1.3,1.3,1.3,14.54,0.0,1.0,17.0,43.0


humaninfluence_attributes_df


,benchmark_catch,surfacewater_abs,groundwater_abs,discharges,abs_agriculture_perc,abs_amenities_perc,abs_energy_perc,abs_environmental_perc,abs_industry_perc,abs_watersupply_perc,num_reservoir,reservoir_cap,reservoir_he,reservoir_nav,reservoir_drain,reservoir_wr,reservoir_fs,reservoir_env,reservoir_nousedata,reservoir_year_first,reservoir_year_last
gauge_id,,,,,,,,,,,,,,,,,,,,,
10002,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10003,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,11500,0.0,0.0,0.0,0.0,0.0,0.0,100.0,NaN,NaN


## File for chalk streams

In [11]:
# Define table name
path = (
    os.path.join(
        camels_gb_datasets_dir,
        "chalk_streams.csv"
    )
)

# Add to attributes data frame list (of names!)
attributes_df_names_list.append("chalk_streams_df")

# Read the file into a DataFrame
chalk_streams_df = pd.read_csv(
    path,
    dtype={attributes_index: 'str'},
    index_col=attributes_index
)

print(attributes_df_names_list[-1])
display(chalk_streams_df.head(3))

chalk_streams_df


,chalk_stream_flag
gauge_id,
10002,False
10003,False
1001,False


## Regroup variables

In [12]:
# List the data frame (🚩 elements will be data frames!)
attributes_df_list = [globals()[name] for name in attributes_df_names_list]

# Length of the list of data frame
n_df = len(attributes_df_list)

## Check on indices

In [13]:
# Use the first data frame to infer the list of item
reference_index = attributes_df_list[0].index

for i, df in enumerate(attributes_df_list):
    assert df.index.equals(reference_index), f"Data frame {attributes_df_names_list[i]} has a different index."

## `full_list_of_fields` aggregation

In [14]:
# Initialization of the aggregation data frame
full_list_of_fields_df = attributes_df_list[0]

# Loop on joining data frames
for i in range(1,n_df):
    
    # Join current data frame with the following
    full_list_of_fields_df = (
        full_list_of_fields_df.
            join(attributes_df_list[i])
    )

### Checks on `full_list_of_fields` aggregation

In [15]:
assert len(full_list_of_fields_df.columns) == sum(len(df.columns) for df in attributes_df_list), (
    "Total number of columns for aggregated data frame does NOT match the sum of columns numerosity of the original data frames"
)

In [18]:
display(full_list_of_fields_df.head(3))

,p_mean,pet_mean,aridity,p_seasonality,frac_snow,high_prec_freq,high_prec_dur,high_prec_timing,low_prec_freq,low_prec_dur,low_prec_timing,inter_high_perc,inter_mod_perc,inter_low_perc,frac_high_perc,frac_mod_perc,frac_low_perc,no_gw_perc,low_nsig_perc,nsig_low_perc,dwood_perc,ewood_perc,grass_perc,shrub_perc,crop_perc,urban_perc,inwater_perc,bares_perc,dom_land_cover,q_mean,runoff_ratio,stream_elas,slope_fdc,baseflow_index,baseflow_index_ceh,hfd_mean,Q5,Q95,high_q_freq,high_q_dur,low_q_freq,low_q_dur,zero_q_freq,gauge_name,gauge_lat,gauge_lon,gauge_easting,gauge_northing,gauge_elev,area,dpsbar,elev_mean,elev_min,elev_10,elev_50,elev_90,elev_max,station_type,flow_period_start,flow_period_end,flow_perc_complete,bankfull_flow,structurefull_flow,q5_uncert_upper,q5_uncert_lower,q25_uncert_upper,q25_uncert_lower,q50_uncert_upper,q50_uncert_lower,q75_uncert_upper,q75_uncert_lower,q95_uncert_upper,q95_uncert_lower,q99_uncert_upper,q99_uncert_lower,quncert_meta,sand_perc,sand_perc_missing,silt_perc,silt_perc_missing,clay_perc,clay_perc_missing,organic_perc,organic_perc_missing,bulkdens,bulkdens_missing,bulkdens_5,bulkdens_50,bulkdens_95,tawc,tawc_missing,tawc_5,tawc_50,tawc_95,porosity_cosby,porosity_cosby_missing,porosity_cosby_5,porosity_cosby_50,porosity_cosby_95,porosity_hypres,porosity_hypres_missing,porosity_hypres_5,porosity_hypres_50,porosity_hypres_95,conductivity_cosby,conductivity_cosby_missing,conductivity_cosby_5,conductivity_cosby_50,conductivity_cosby_95,conductivity_hypres,conductivity_hypres_missing,conductivity_hypres_5,conductivity_hypres_50,conductivity_hypres_95,root_depth,root_depth_missing,root_depth_5,root_depth_50,root_depth_95,soil_depth_pelletier,soil_depth_pelletier_missing,soil_depth_pelletier_5,soil_depth_pelletier_50,soil_depth_pelletier_95,benchmark_catch,surfacewater_abs,groundwater_abs,discharges,abs_agriculture_perc,abs_amenities_perc,abs_energy_perc,abs_environmental_perc,abs_industry_perc,abs_watersupply_perc,num_reservoir,reservoir_cap,reservoir_he,reservoir_nav,reservoir_drain,reservoir_wr,reservoir_fs,reservoir_env,reservoir_nousedata,reservoir_year_first,reservoir_year_last,chalk_stream_flag
gauge_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10002,2.29,1.26,0.55,-0.03,0.03,13.96,1.16,son,207.89,3.17,jja,0.0,18.53,0.0,0.0,0.00,23.72,57.67,0.0,0.08,3.89,5.41,42.48,6.16,40.37,1.72,0.13,0.01,Grass and Pasture,1.30,0.57,1.36,1.94,0.67,0.63,131.33,0.31,3.48,2.17,1.35,7.41,7.52,0.0,Ugie at Inverugie,57.53,-1.83,410113,848516,8.5,325.72,42.3,87.0,9.5,38.6,84.1,140.1,233.7,VA,1971-02-01,2015-09-30,99.25,100.0,NaN,NaN,NaN,11.95,-12.37,9.55,-9.40,9.35,-9.61,9.02,-9.23,8.25,-9.50,Calculated discharge uncertainties,40.98,0.00,35.60,0.00,23.42,0.00,5.22,0.00,1.20,0.00,0.28,1.36,1.42,116.79,0.00,55.12,110.37,235.65,43.81,0.00,42.75,43.82,44.36,51.13,0.00,44.0,47.0,81.0,1.34,0.00,1.15,1.17,2.31,1.49,0.00,1.27,1.34,3.13,1.20,0.00,0.5,1.3,1.3,13.21,0.0,1.0,2.0,41.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
10003,2.31,1.25,0.54,-0.01,0.03,13.24,1.17,son,209.89,3.20,jja,0.0,13.14,0.0,0.0,0.00,23.03,63.72,0.0,0.12,4.74,3.06,36.88,0.31,54.06,1.04,0.06,0.03,Crops,1.33,0.57,1.31,1.95,0.72,0.73,139.30,0.32,3.40,0.90,1.32,10.81,10.00,0.0,Ythan at Ellon,57.36,-2.09,394684,830370,3.8,532.29,55.1,108.0,3.5,54.1,106.1,160.8,380.4,VA,1983-05-19,2015-09-30,71.93,80.0,NaN,NaN,NaN,12.58,-12.46,8.73,-8.68,11.69,-11.89,8.00,-8.06,NaN,NaN,Calculated discharge uncertainties,43.79,0.00,37.96,0.00,18.25,0.00,1.58,0.00,1.29,0.00,1.10,1.36,1.42,88.47,0.00,55.12,102.07,110.37,43.61,0.00,42.75,43.74,44.36,47.54,0.00,44.0,47.0,52.0,1.58,0.00,1.17,1.39,2.31,1.50,0.00,1.27,1.36,2.28,1.01,0.00,0.5,1.3,1.3,7.52,0.0,1.0,1.0,30.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1001,2.65,1.15,0.44,-0.15,0.03,10.98,1.11,son,179.87,2.93,jja,0.0,10.20,0.0,0.0,2.24,0.00,8

In [19]:
print(f"Number of columns for `full_list_of_fields` data frame: {len(full_list_of_fields_df.columns)}")

Number of columns for `full_list_of_fields` data frame: 146


### Save `full_list_of_fields`

In [20]:
# Define path to save
path = os.path.join(
        camels_gb_data_attributes_aggr_dir,
        "full_list_of_fields.csv"
)

# Save
full_list_of_fields_df.to_csv(path)

## `fundamental` aggregation

In [21]:
# First set of columns
curr_columns_set = attributes_fundamental_fields[attributes_df_names_list[0]]

# Initialization of the aggregation data frame
fundamental_df = attributes_df_list[0][curr_columns_set]

# Loop on joining data frames
for i in range(1,n_df):
    
    # Join current data frame with the following
    fundamental_df = (
        fundamental_df.
            join(
                attributes_df_list[i][
                    attributes_fundamental_fields[attributes_df_names_list[i]]
                ]
            )
    )

### Checks on `fundamental` aggregation

In [22]:
assert len(fundamental_df.columns) == sum(len(fields_list) for fields_list in attributes_fundamental_fields.values()), (
    "Total number of columns for aggregated data frame does NOT match the sum of columns inside the dictionary stating fundamental attributes fields"
)

In [24]:
display(fundamental_df.head(3))

,dwood_perc,ewood_perc,grass_perc,shrub_perc,crop_perc,urban_perc,inwater_perc,bares_perc,baseflow_index,gauge_name,gauge_lat,gauge_lon,gauge_elev,area,dpsbar,elev_mean,elev_min,elev_10,elev_50,elev_90,elev_max,bankfull_flow,sand_perc,silt_perc,clay_perc,organic_perc,surfacewater_abs,groundwater_abs,discharges,num_reservoir,reservoir_cap,chalk_stream_flag
gauge_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10002,3.89,5.41,42.48,6.16,40.37,1.72,0.13,0.01,0.67,Ugie at Inverugie,57.53,-1.83,8.5,325.72,42.3,87.0,9.5,38.6,84.1,140.1,233.7,100.0,40.98,35.60,23.42,5.22,NaN,NaN,NaN,0,0,False
10003,4.74,3.06,36.88,0.31,54.06,1.04,0.06,0.03,0.72,Ythan at Ellon,57.36,-2.09,3.8,532.29,55.1,108.0,3.5,54.1,106.1,160.8,380.4,80.0,43.79,37.96,18.25,1.58,NaN,NaN,NaN,0,0,False
1001,0.41,12.37,39.84,41.89,1.78,0.21,3.36,0.30,0.49,Wick at Tarroul,58.48,-3.27,13.0,158.18,29.9,80.0,12.9,30.6,78.8,135.2,241.4,31.2,39.25,31.97,28.79,19.32,NaN,NaN,NaN,1,11500,False


In [25]:
print(f"Number of columns for `fundamental` data frame: {len(fundamental_df.columns)}")

Number of columns for `fundamental` data frame: 32


### API catchments set

#### Retrieve full list of `nrfaStationID` codes

In [26]:
# _______________________________________
# Retrieve the list of available stations
url = f"{APIBaseURL}/hydrology/id/stations.json?_limit=100000"

# Call to review ALL the stations
json_data = api_call(url)
print(f"N. of stations found:\t{len(json_data['items'])}")


# ___________________________________________
# Set dataFrames to collect station meta date

# Imported from the full list of stations meta data
full_stations_OpenAPI_df = pd.DataFrame({
    '@id': [item['@id'] for item in json_data['items']],
    'label': [item['label'] for item in json_data['items']]
})

# Create api for specific call on station details
full_stations_OpenAPI_df['query_for_station_specifics'] = full_stations_OpenAPI_df['@id'] + ".json"


# ________________________
# Retrieve `nrfaStationID`

# Call `get_nrfaStationID` for each station
full_stations_OpenAPI_df['nrfaStationID'] = full_stations_OpenAPI_df['query_for_station_specifics'].apply(get_nrfaStationID)

# Quick overview
full_stations_OpenAPI_df['nrfaStationID'].value_counts()

N. of stations found:	9194


nrfaStationID
No nrfaStationID found    8348
28007                        2
39042                        2
33007                        1
55003                        1
                          ... 
27086                        1
38016                        1
72014                        1
27060                        1
28047                        1
Name: count, Length: 845, dtype: int64

#### Manage problematic cases

In [27]:
# Display duplicates
print(
    full_stations_OpenAPI_df
    .loc[lambda df: df['nrfaStationID'] != 'No nrfaStationID found']
    .loc[lambda df: df['nrfaStationID'].isin(df['nrfaStationID'].value_counts()[lambda x: x != 1].index)]
    .sort_values(by='nrfaStationID')
    .values
)

[['http://environment.data.gov.uk/hydrology/id/stations/4a8d8712-14de-4faf-8f75-e61eb2bd3128'
  'Shardlow'
  'http://environment.data.gov.uk/hydrology/id/stations/4a8d8712-14de-4faf-8f75-e61eb2bd3128.json'
  '28007']
 ['http://environment.data.gov.uk/hydrology/id/stations/16b72267-fdbb-4e1c-9f45-c7bb401d3804'
  'Shardlow'
  'http://environment.data.gov.uk/hydrology/id/stations/16b72267-fdbb-4e1c-9f45-c7bb401d3804.json'
  '28007']
 ['http://environment.data.gov.uk/hydrology/id/stations/ef76b105-7412-45fc-976d-972c1f7e185a'
  'Lechlade'
  'http://environment.data.gov.uk/hydrology/id/stations/ef76b105-7412-45fc-976d-972c1f7e185a.json'
  '39042']
 ['http://environment.data.gov.uk/hydrology/id/stations/155ceefb-3ab6-436d-90da-6187f554f20b'
  'Thorpe Lakes'
  'http://environment.data.gov.uk/hydrology/id/stations/155ceefb-3ab6-436d-90da-6187f554f20b.json'
  '39042']]


In [28]:
# Dropping problematic `nrfaStationID``
unique_stations_OpenAPI_df = (
    full_stations_OpenAPI_df
        .loc[lambda df: df['nrfaStationID'] != 'No nrfaStationID found']
        .loc[lambda df: df['nrfaStationID'].isin(df['nrfaStationID'].value_counts()[lambda x: x == 1].index)]
)

#### Join API and CAMELS_GB ***stations***

In [29]:
# INNER JOIN with attributes data frame
print(f"N. of catchments in attributes table BEFORE the merge:\t{fundamental_df.shape[0]}")

fundamental_api_df = (
    fundamental_df
        .merge(
            unique_stations_OpenAPI_df,
            how='inner',
            left_index=True,
            right_on='nrfaStationID'
        )
        .set_index('nrfaStationID')
        .rename_axis(attributes_index)
)

print(f"N. of catchments remained AFTER the merge:\t\t{fundamental_api_df.shape[0]}")

N. of catchments in attributes table BEFORE the merge:	671
N. of catchments remained AFTER the merge:		419


#### Derive URL for time series (`measures`)

In [30]:
# Derive station code
fundamental_api_df['station_code'] = fundamental_api_df['@id'].str.split('/').str[-1]

# Derive URL for time series query
fundamental_api_df['query_for_station_specifics'] = fundamental_api_df['station_code'].apply(get_flow_timeseries_url)

display(fundamental_api_df.head())

,dwood_perc,ewood_perc,grass_perc,shrub_perc,crop_perc,urban_perc,inwater_perc,bares_perc,baseflow_index,gauge_name,gauge_lat,gauge_lon,gauge_elev,area,dpsbar,elev_mean,elev_min,elev_10,elev_50,elev_90,elev_max,bankfull_flow,sand_perc,silt_perc,clay_perc,organic_perc,surfacewater_abs,groundwater_abs,discharges,num_reservoir,reservoir_cap,chalk_stream_flag,@id,label,query_for_station_specifics,station_code
gauge_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101002,6.20,0.30,42.78,0.38,46.34,3.28,0.49,0.56,0.68,Medina at Shide,50.69,-1.29,10.4,30.61,80.0,60.0,9.4,26.8,56.8,98.0,193.2,17.8,70.99,15.36,13.66,0.43,0.006,0.023,0.018,0,0,True,http://environment.data.gov.uk/hydrology/id/st...,Shide,https://environment.data.gov.uk/hydrology/id/m...,69f4b3e5-a487-4769-aded-0d72062428e7
101005,4.60,0.12,59.24,0.25,31.08,4.78,0.05,0.00,0.62,Eastern Yar at Budbridge,50.65,-1.25,17.2,24.31,87.0,92.0,17.2,32.9,84.0,157.2,234.6,NaN,71.01,15.56,13.45,0.45,0.001,0.015,0.053,0,0,True,http://environment.data.gov.uk/hydrology/id/st...,Budbridge,https://environment.data.gov.uk/hydrology/id/m...,c10d61a3-fc5d-4d96-bf9f-57a97b6256c0
22001,5.87,9.60,59.92,11.13,12.49,0.95,0.13,0.00,0.51,Coquet at Morwick,55.33,-1.63,5.2,578.25,110.0,225.0,5.2,79.3,192.8,420.0,774.5,175.0,45.16,31.88,22.96,4.13,0.000,0.006,0.004,0,0,False,http://environment.data.gov.uk/hydrology/id/st...,Morwick,https://environment.data.gov.uk/hydrology/id/m...,3df7a9c3-d40c-4781-a885-5f6b7abdb86a
22006,5.29,0.76,45.06,0.13,44.22,3.85,0.27,0.57,0.43,Blyth at Hartford Bridge,55.11,-1.62,24.6,273.67,31.9,115.0,24.0,57.8,110.5,185.4,265.2,190.0,37.67,35.23,27.10,0.54,0.065,0.000,0.002,0,0,False,http://environment.data.gov.uk/hydrology/id/st...,Hartford Bridge,https://environment.data.gov.uk/hydrology/id/m...,c0744418-231d-4953-be28-bbef415aea01
22007,6.57,13.89,56.64,5.43,16.66,0.57,0.45,0.00,0.44,Wansbeck at Mitford,55.17,-1.73,31.4,282.01,50.8,182.0,30.8,93.9,176.1,283.1,442.2,NaN,39.41,34.78,25.81,1.56,0.066,0.000,0.000,1,3279,False,http://environment.data.gov.uk/hydrology/id/st...,Mitford,https://environment.data.gov.uk/hydrology/id/m...,9af510f4-5629-465b-81a9-855274a8750a


### Save `fundamental`

In [33]:
# Define path to save
path = os.path.join(
        camels_gb_data_attributes_aggr_dir,
        "fundamental.csv"
)

# Save
fundamental_api_df.to_csv(path)

# Timeseries aggregations

## `timeseries` aggregation

In [34]:
# Create an empty data frame to store timeseries
timeseries_df = pd.DataFrame()

# Create an empty dictionary to store potential aggregation issues 
timeseries_aggr_issues_dict = {}

# Create counter for unmatched catchments
i = 1

print(f"Processing catchments: ", end="")

# Loop through all files in the directory for camels-gb time series
for filename in os.listdir(camels_gb_data_timeseries_dir):
    
    # Define current full path
    path = os.path.join(camels_gb_data_timeseries_dir, filename)
    
    # Check if it's a file and has the desired extension
    if os.path.isfile(path) and filename.endswith(timeseries_files_ext):
    
        # Extract catchmentID
        match = re.search(catchmentID_pattern, filename)
        
        # When a match is found..
        if match:
            catchmentID = match.group(1)
            print(f"ID: {catchmentID} ...   ", end="")
        
            # Read the file into a DataFrame
            df = pd.read_csv(
                path,
                parse_dates=[date_field]
            )
            
            # Convert from Timestamp to datetime.date
            df[date_field] = df[date_field].dt.date

            # Add catchment ID (as first column)
            df.insert(0, 'catchmentID', catchmentID)
            
            # Aggregation
            timeseries_df = pd.concat([timeseries_df, df], ignore_index=True)
            
            print("Ok!  ", end="")

        #..otherwise notify which is the problematic file..
        #..and skip to the following
        else:
            print(f"Catchment ID not found for file {filename}  ", end="")
            timeseries_aggr_issues_dict[f"Unknown_{i}"] = {
                    "File": path,
                    "Issue": "File skipped because not able to retrieve catchmentID from file name"
            }
            i += 1
            continue

            
# Final order by `catchmentID`
print("\nFinal sorting..")
timeseries_df.sort_values(
    by=["catchmentID", date_field],
    inplace=True,
    ignore_index=True
)

print("Aggregation complete!")

# Display issues, if any
if timeseries_aggr_issues_dict:
    print(f"Please have a look at the following issues during the aggregation process:\n{timeseries_aggr_issues_dict}")

Processing catchments: ID: 39008 ...   Ok!  ID: 41019 ...   Ok!  ID: 90003 ...   Ok!  ID: 45009 ...   Ok!  ID: 52004 ...   Ok!  ID: 38017 ...   Ok!  ID: 53018 ...   Ok!  ID: 21012 ...   Ok!  ID: 50006 ...   Ok!  ID: 60002 ...   Ok!  ID: 33021 ...   Ok!  ID: 54027 ...   Ok!  ID: 44001 ...   Ok!  ID: 26008 ...   Ok!  ID: 27042 ...   Ok!  ID: 26009 ...   Ok!  ID: 54080 ...   Ok!  ID: 55026 ...   Ok!  ID: 84008 ...   Ok!  ID: 84004 ...   Ok!  ID: 74005 ...   Ok!  ID: 36012 ...   Ok!  ID: 76014 ...   Ok!  ID: 28040 ...   Ok!  ID: 41012 ...   Ok!  ID: 15016 ...   Ok!  ID: 29003 ...   Ok!  ID: 49001 ...   Ok!  ID: 88001 ...   Ok!  ID: 84016 ...   Ok!  ID: 10003 ...   Ok!  ID: 11004 ...   Ok!  ID: 33020 ...   Ok!  ID: 53005 ...   Ok!  ID: 29009 ...   Ok!  ID: 84022 ...   Ok!  ID: 38003 ...   Ok!  ID: 57008 ...   Ok!  ID: 17001 ...   Ok!  ID: 41029 ...   Ok!  ID: 45012 ...   Ok!  ID: 34012 ...   Ok!  ID: 54008 ...   Ok!  ID: 21006 ...   Ok!  ID: 71004 ...   Ok!  ID: 28008 ...   Ok!  ID: 16003 .

### Checks on `timeseries_df` aggregation

In [35]:
display(timeseries_df)

,catchmentID,date,precipitation,pet,temperature,discharge_spec,discharge_vol,peti,humidity,shortwave_rad,longwave_rad,windspeed
0,10002,1970-10-01,4.67,1.15,9.38,NaN,NaN,1.43,6.05,71.60,323.22,7.38
1,10002,1970-10-02,1.70,1.41,8.27,NaN,NaN,1.76,5.45,115.58,287.59,10.39
2,10002,1970-10-03,4.28,1.27,9.17,NaN,NaN,1.52,5.56,76.19,315.95,5.92
3,10002,1970-10-04,2.41,0.62,10.75,NaN,NaN,0.82,7.40,45.54,345.59,6.39
4,10002,1970-10-05,0.02,1.44,10.39,NaN,NaN,1.48,6.21,109.42,298.20,7.96
...,...,...,...,...,...,...,...,...,...,...,...,...
11028551,97002,2015-09-26,0.09,0.59,11.50,0.56,2.67,0.68,6.92,27.10,350.36,2.92
11028552,97002,2015-09-27,0.00,0.96,11.09,0.53,2.52,0.96,6.66,99.39,316.69,2.51
11028553,97002,2015-09-28,0.00,0.85,10.79,0.50,2.40,0.85,6.61,68.27,333.35,2.80
11028554,97002,2015-09-29,0.00,1.39,12.07,0.52,2.51,1.39,6.48,144.19,287.70,3.32


### Download API time series with `Good` and `Complete` quality flags

#### Download

In [36]:
# Flow timeseries API data frame
discharge_api_df = pd.DataFrame()

# Loop through the DataFrame
for index, row in fundamental_api_df.iterrows():

    # Create a DataFrame with the data from the API call
    curr_df = pd.DataFrame(
        api_call(
            row['query_for_station_specifics']
        )
        ['items']
    )

    # Add the station code
    curr_df.insert(0, 'catchmentID', index)

    # Concatenate with the main data frame
    discharge_api_df = pd.concat(
        [
            discharge_api_df,
            curr_df
        ],
        axis=0
    )

# Make-up
discharge_api_df.rename(columns={'value': label_field}, inplace=True)
discharge_api_df['date'] = pd.to_datetime(discharge_api_df['date']).dt.date

#### `Good` and `Complete` only

In [37]:
discharge_api_df = (
    discharge_api_df[
        (discharge_api_df['completeness'] == 'Complete') &
            (discharge_api_df['quality'] == 'Good')
    ]
)

display(discharge_api_df.head())

,catchmentID,measure,date,dateTime,discharge_vol,valid,invalid,missing,completeness,quality,qcode
0,101002,{'@id': 'http://environment.data.gov.uk/hydrol...,1997-03-01,1997-03-01T09:00:00,0.319,10000,0,0,Complete,Good,NaN
1,101002,{'@id': 'http://environment.data.gov.uk/hydrol...,1997-03-02,1997-03-02T09:00:00,0.314,10000,0,0,Complete,Good,NaN
2,101002,{'@id': 'http://environment.data.gov.uk/hydrol...,1997-03-03,1997-03-03T09:00:00,0.397,10000,0,0,Complete,Good,NaN
3,101002,{'@id': 'http://environment.data.gov.uk/hydrol...,1997-03-04,1997-03-04T09:00:00,0.866,10000,0,0,Complete,Good,NaN
4,101002,{'@id': 'http://environment.data.gov.uk/hydrol...,1997-03-05,1997-03-05T09:00:00,0.719,10000,0,0,Complete,Good,NaN


In [38]:
# Numerosity BEFORE merge
print(f"CAMELS-GB:\t{timeseries_df.shape[0]:,}")
print(f"API:\t\t{discharge_api_df.shape[0]:,}")

CAMELS-GB:	11,028,556
API:		5,438,701


### Join API with CAMELS_GB ***timeseries***

In [39]:
# `discharge_vol` values check
timeseries_api_df = (
    timeseries_df
    .merge(
        discharge_api_df[['catchmentID', 'date', label_field]],
        how='left',
        left_on=['catchmentID', date_field],
        right_on=['catchmentID', date_field],
        suffixes=('_files', '_api')
    )
)

# Drop NaNs
timeseries_api_df.dropna(inplace=True)

# Make-up
timeseries_api_df.rename(
    columns={
        f"{label_field}_api": label_field
    },
    inplace=True
)

timeseries_api_df.reset_index(
    inplace=True,
    drop=True
)
display(timeseries_api_df.head())

,catchmentID,date,precipitation,pet,temperature,discharge_spec,discharge_vol_files,peti,humidity,shortwave_rad,longwave_rad,windspeed,discharge_vol
0,101002,1997-03-01,0.26,0.70,8.31,0.90,0.32,0.93,6.11,73.68,320.09,6.23,0.319
1,101002,1997-03-02,0.10,1.42,9.55,0.89,0.31,1.68,5.64,89.11,315.49,6.51,0.314
2,101002,1997-03-03,24.15,0.53,5.84,1.12,0.40,0.65,4.98,51.62,320.09,2.75,0.397
3,101002,1997-03-04,3.06,0.15,8.56,2.44,0.87,0.20,6.73,31.00,336.67,3.50,0.866
4,101002,1997-03-05,2.00,0.67,8.52,2.03,0.72,0.83,6.11,59.23,330.95,4.14,0.719


In [40]:
# N. catchments after API's quality checks
print(f"N. of catchments with API's quality checks:\t{timeseries_api_df['catchmentID'].nunique()}")

N. of catchments with API's quality checks:	410


### Split of catchments for not continuous time series

In [41]:
# Create en empty list to store catchment groups with too short time series
catchmentsID_ts_too_short_list = []

def split_timeseries(group):

    # Retrieving current catchment ID
    curr_catchmentID = group['catchmentID'].iloc[0]

    # Calculate the difference in days
    group[f"{date_field}_diff"] = group[date_field].diff().dt.days

    # Fill NaN values with 0
    group[f"{date_field}_diff"] = group[f"{date_field}_diff"].fillna(0)

    # Determine if the date is the same as the previous day
    group[f"{date_field}_consecutive_day"] = group[f"{date_field}_diff"] > 1

    # Calculate the cumulative sum to create groups
    group[f"{date_field}_group"] = group[f"{date_field}_consecutive_day"].cumsum()

    # Convert group into string
    group[f"{date_field}_group"] = (
        group[f"{date_field}_group"]
            .astype(str)
            .str
            .zfill(2)
    )

    # Calculate the size of each group
    group_sizes = group[f"{date_field}_group"].value_counts()

    # Save list of catchments with too short time series
    (
        catchmentsID_ts_too_short_list.extend([
            f"{curr_catchmentID}-{x}" for x in
                group_sizes[
                    group_sizes < min_timeseries_length
                ]
                .index
                .to_list()
        ])
    )

    # Remove `group` with less than `min_timeseries_length`
    filtered_group  = (
        group[
            group[f"{date_field}_group"]
                .isin(
                    group_sizes[
                        group_sizes >= min_timeseries_length
                    ].index
                )
        ]
    )

    return filtered_group

# Apply the function to each sensor
timeseries_api_df = (
    timeseries_api_df
        .groupby('catchmentID')
        .apply(split_timeseries)
        .reset_index(drop=True)
)

display(timeseries_api_df.head())

/var/tmp/ipykernel_79052/3173399142.py:62: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(split_timeseries)


,catchmentID,date,precipitation,pet,temperature,discharge_spec,discharge_vol_files,peti,humidity,shortwave_rad,longwave_rad,windspeed,discharge_vol,date_diff,date_consecutive_day,date_group
0,101002,1997-03-01,0.26,0.70,8.31,0.90,0.32,0.93,6.11,73.68,320.09,6.23,0.319,0.0,False,00
1,101002,1997-03-02,0.10,1.42,9.55,0.89,0.31,1.68,5.64,89.11,315.49,6.51,0.314,1.0,False,00
2,101002,1997-03-03,24.15,0.53,5.84,1.12,0.40,0.65,4.98,51.62,320.09,2.75,0.397,1.0,False,00
3,101002,1997-03-04,3.06,0.15,8.56,2.44,0.87,0.20,6.73,31.00,336.67,3.50,0.866,1.0,False,00
4,101002,1997-03-05,2.00,0.67,8.52,2.03,0.72,0.83,6.11,59.23,330.95,4.14,0.719,1.0,False,00


### Save `timeseries_df`

In [42]:
# Define path to save
path = os.path.join(
        camels_gb_data_timeseries_aggr_dir,
        "timeseries.csv"
)

# Save
timeseries_api_df.to_csv(
    path,
    index=False
)

# Metadata

## Store parameters used for aggregations

In [43]:
# Create dictionary
aggr_parameters_dict = {
    "camels_gb_use_case_dir": camels_gb_use_case_dir,
    "camels_gb_datasets_dir": camels_gb_datasets_dir,
    "camels_gb_bronze_dir": camels_gb_bronze_dir,
    "camels_gb_silver_dir": camels_gb_silver_dir,
    "camels_gb_data_attributes_dir": camels_gb_data_attributes_dir,
    "camels_gb_data_attributes_aggr_dir": camels_gb_data_attributes_aggr_dir,
    "camels_gb_data_timeseries_dir": camels_gb_data_timeseries_dir,
    "camels_gb_data_timeseries_aggr_dir": camels_gb_data_timeseries_aggr_dir,
    "attributes": {
        "attributes_files_ext": attributes_files_ext,
        "attributes_index": attributes_index,
        "aggregations": {
            "fundamental": attributes_fundamental_fields
        }
    },
    "timeseries": {
        "timeseries_files_ext": timeseries_files_ext,
        "catchmentID_pattern": catchmentID_pattern,
        "date_field": date_field,
        "label_field": label_field,
        "timeseries_aggr_issues_dict": timeseries_aggr_issues_dict
    }
}

# Store dictionary
with open(
    os.path.join(
        camels_gb_use_case_dir,
        'aggr_parameters_dict.pkl'
    ),
    'wb'
) as f:
    pickle.dump(aggr_parameters_dict, f)

## Store dictionary with time range per `catchment`-`group` time series

In [44]:
# Derive statistics about length (as data frame)
length_df = (
    timeseries_api_df
        .groupby(
            ['catchmentID', f"{date_field}_group"]
        )
        .agg(
            start_date=(date_field, 'min'),
            end_date=(date_field, 'max'),
            length=(date_field, 'count')
        )
        .reset_index()
)


# Create a dictionary properly formatted
length_dict = length_df.set_index(['catchmentID', f"{date_field}_group"]).to_dict(orient='index')
catchmentID_time_ranges_dict = {
    f"{key[0]}-{key[1]}": value
    for key, value in length_dict.items()
}


# Store
with open(
    os.path.join(
        camels_gb_use_case_dir,
        'catchmentID_time_ranges_dict.pkl'
    ),
    'wb'
) as f:
    pickle.dump(catchmentID_time_ranges_dict, f)

## Store list for too short `catchment`-`group` time series

In [45]:
with open(
    os.path.join(
        camels_gb_use_case_dir,
        f"catchmentsID_ts_shorter_than_{min_timeseries_length}.pkl"
    ),
    'wb'
) as f:
    pickle.dump(catchmentsID_ts_too_short_list, f)